In [1]:
import pymysql.cursors
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO
from Bio.Phylo.Applications import FastTreeCommandline
from Bio import Phylo
%matplotlib inline
import scipy.cluster.hierarchy as cl
import scipy.spatial.distance as ssd
import matplotlib
import matplotlib.pyplot as plt
from Bio import Entrez, SeqIO
import time
from termcolor import colored

Entrez.email = "vincent.deruaz@master.hes-so.ch"


/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
class sea_inphinity():
    def __init__(self, db_name):
        self.db_name = db_name
        self.connection()
        self.verbose = True
        self.out_dir = './out/'
        
    def print_(self, string):
        if self.verbose:
            print(string)
        
    def connection(self):
        # Connect to the database
        self.connection = pymysql.connect(host='localhost',
                                     user='root',
                                     password='',
                                     db=self.db_name,
                                     charset='utf8mb4',
                                     cursorclass=pymysql.cursors.DictCursor)
        
    def simple_execute(self, sql, verbose):
        try:
            with self.connection.cursor() as cursor:
                res = cursor.execute(sql)

                if verbose:
                    self.print_(res)
                    
                return res

            self.connection.commit()
        finally:
            pass
        
    def get_list_pham(self, limit):
        cur = self.connection.cursor()

        if limit == -1:
            sql = "SELECT * FROM pham"
        else:
            sql = "SELECT * FROM pham LIMIT %d" % (limit)
        cur.execute(sql)

        for row in cur:
            self.print_(row)

        return cur
        
    
    def get_list_name_pham(self, limit):
        cur = self.connection.cursor()

        if limit == -1:
            sql = "SELECT name FROM pham GROUP BY name"
        else:
            sql = "SELECT name FROM pham GROUP BY name LIMIT %d" % (limit)
        cur.execute(sql)
        
        list_name = []
        for name in list(cur):
            list_name.append(name['name'])

        return list_name
    
    def get_specific_pham(self, name):
        cur = self.connection.cursor()

        cur.execute("SELECT GeneID FROM pham WHERE name = %s", (name))

        return cur
    
    def get_list_genes(self, gene_ids, verbose):
        cur = self.connection.cursor()
        
        sql = "SELECT * FROM gene WHERE GeneID IN ('%s');" % ( "','".join(gene_ids))
        
        cur.execute(sql)

        if verbose:
            for row in cur:
                self.print_(row)

        return list(cur.fetchall())
    
    def get_phage(self, phage_id, verbose):
        cur = self.connection.cursor()
        
        sql = "SELECT * FROM phage WHERE PhageID = '%s';" % (phage_id)
        
        cur.execute(sql)

        if verbose:
            for row in cur:
                self.print_(row)

        return list(cur.fetchall())
    
    def get_phage_from_gene_id(self, gene_id):
        phage = sea_inphinity.get_phage(sea_inphinity.get_list_genes([gene_id], False)[0]['PhageID'], False)[0]
        return phage
    
    def build_tree(self,pham):
        genes = self.get_genes_from_a_pham(pham)
        self.create_fasta(genes)
        self.align_muscle()
        self.compute_tree()
        self.prepare_tree_fig()
    
    def get_genes_from_a_pham(self, pham):
        pham_list = []
        pham = sea_inphinity.get_specific_pham(pham)
        for id in pham:
            pham_list.append(id['GeneID'])
            self.print_(id)
        #print(pham_list)
        genes = sea_inphinity.get_list_genes(pham_list, False)
        
        return genes
        
    def create_fasta(self, genes):
        print('Creation of the FASTA file')
        fasta = open("%sfasta.fa" % (self.out_dir), "w")
        self.print_("Number of Genes: %d" % (len(genes)))
        for gene in genes:
            GeneID = gene['GeneID']
            name = gene['Name']
            description = ">%s - %s" % (GeneID, name)

            translation = gene['translation']

            self.print_(description)
            self.print_(translation)

            fasta.write(description)
            fasta.write('\n')
            fasta.write(translation)
            fasta.write('\n')

        fasta.close()
        
    def align_muscle(self):
        print('Alignment with MUSCLE')
        muscle_loc = r'/home/pa/work/muscle3.8.31_i86linux64' # modifier si nécessaire

        muscle_cline = MuscleCommandline(cmd=muscle_loc,input='%sfasta.fa' % (self.out_dir),out='%sout.aln' % (self.out_dir),clwstrict=True)
        stdout, stderr = muscle_cline()

        muscle_align = AlignIO.read('%sout.aln' % (self.out_dir),'clustal')
        self.print_(muscle_align)
        
    def compute_tree(self):
        print('Compute tree')
        AlignIO.convert('%sout.aln' % (self.out_dir),'clustal','%sintermediate.phy' % (self.out_dir), 'phylip-relaxed')

        cmd_fasttree = r'fasttree'
        fasttree_cmdline = FastTreeCommandline(cmd=cmd_fasttree,fastest=True, \
                                               input='%sintermediate.phy' % (self.out_dir),out='%stree.tre' % (self.out_dir))
        out_log, err_log = fasttree_cmdline()

        self.print_('Out Log:')
        self.print_(out_log)

        self.print_('Error Log')
        self.print_(err_log)
        
        self.tree = Phylo.read('%stree.tre' % (self.out_dir), 'newick')
        
    def prepare_tree_fig(self):
        print('Prepare Tree Fig')
        self.dmat = []
        self.leaves = [str(cladit) for k,cladit in enumerate(self.tree.get_terminals())]
        for l1,leave1 in enumerate(self.leaves):
            d = []
            for l2,leave2 in enumerate(self.leaves):
                d.append(self.tree.distance(leave1,leave2))
            self.dmat.append(d)
            
        
        
    def draw_tree(self):
        print('Draw Tree')
        Z = cl.linkage(ssd.squareform(self.dmat),method='average',metric='euclidean')
        fig = plt.figure(num=None,figsize=(30,25),dpi=250)
        dendro=cl.dendrogram(Z,labels=self.leaves,color_threshold=0.06,leaf_rotation=90,leaf_font_size=10)
        
        self.plt_tree = plt
        
        self.plt_tree.show()
        
    def print_informations_on_phage(self, gene_id):
        phage = sea_inphinity.get_phage_from_gene_id(gene_id)
        color = 'blue'
        
        source = phage['HostStrain']
        if source == 'None':
            color = 'red'
            
            if phage['Accession'] != '':
                phage_ = phage['Accession']
            else:
                color = 'yellow'
                phage_ = phage['Name'].split('-')[0]
            
            source = sea_inphinity.get_host_from_genbank(phage_)
            print(source)
        
        print("******************************** \
            \nGene ID: %s \nPhage ID: %s \nPhage Name: %s \nPhage Accession: %s" \
            % (gene_id, phage['PhageID'], phage['Name'], phage['Accession']))
        print(colored("HostStrin: %s" % (source), color))        
        print("********************************")
                  
                    
        
    def print_informations_on_phages(self, gene_ids):
        print('===========================')
        print(colored('Blue: Host come from the original database', 'blue'))
        print(colored('Red: Host come from GenBank', 'red'))
        print(colored('Yellow: DRAFT', 'yellow'))
        print('===========================')
        
        for gene_id in gene_ids:
            self.print_informations_on_phage(gene_id)
            
    def get_host_from_genbank(self, genome_id):
        
        try:
            record = Entrez.efetch(db="nuccore", id=genome_id, rettype="gb", retmode="text")

            filename = 'out/genBankRecord.gb'
            with open(filename, 'w') as f:
                f.write(record.read())
            parsed_gb_file = next(SeqIO.parse(filename, "genbank"))

            #print(parsed_gb_file)
            return parsed_gb_file.annotations["source"]
        
        except:
            return 'Not Found'
            

    
sea_inphinity = sea_inphinity('sea')

## Listing Pham

In [ ]:
list_name = sea_inphinity.get_list_name_pham(-1)
print(list_name)

## Testing on Pham 2799

In [4]:
sea_inphinity.verbose = False
sea_inphinity.build_tree('2799')
sea_inphinity.draw_tree()

Creation of the FASTA file
Alignment with MUSCLE


ApplicationError: Non-zero return code 127 from '/home/pa/work/muscle3.8.31_i86linux64 -in ./out/fasta.fa -out ./out/out.aln -clwstrict', message '/bin/sh: 1: /home/pa/work/muscle3.8.31_i86linux64: not found'

### Analyze

In [ ]:
print(sea_inphinity.tree.get_terminals())

In [ ]:
Clade(branch_length=0.02764, name='663557_12')
Clade(branch_length=0.0, name='Backyardigan-DRAFT_gp12')
Clade(branch_length=0.0, name='Wile-DRAFT_gp11')

In [ ]:
Clade(branch_length=0.00623, name='Vix-DRAFT_gp11')
Clade(branch_length=0.0, name='Microwolf-DRAFT_gp14')
Clade(branch_length=0.0, name='205870_14')

In [ ]:
print(sea_inphinity.get_list_genes(['Vix-DRAFT_gp11'], True))

In [ ]:
sea_inphinity.print_informations_on_phage('Vix-DRAFT_gp11')

In [ ]:
sea_inphinity.print_informations_on_phages(['Vix-DRAFT_gp11', 'Microwolf-DRAFT_gp14', '205870_14'])

In [ ]:
sea_inphinity.print_informations_on_phages(['663557_12', 'Backyardigan-DRAFT_gp12', 'Wile-DRAFT_gp11'])

In [ ]:
sea_inphinity.print_informations_on_phages(sea_inphinity.leaves)

## Checking Theory

In [54]:
sea_inphinity.verbose = False
sea_inphinity.build_tree('2799')
sea_inphinity.print_informations_on_phages(sea_inphinity.leaves)

Creation of the FASTA file
Alignment with MUSCLE
Compute tree
Prepare Tree Fig
Blue: Host come from the original database
Red: Host come from GenBank
Yellow: DRAFT
Mycobacterium phage Pukovnik
********************************             
Gene ID: 540068_15 
Phage ID: 540068 
Phage Name: Pukovnik 
Phage Accession: EU744250.1
HostStrin: Mycobacterium phage Pukovnik
********************************
Not Found
********************************             
Gene ID: Turbido-DRAFT_gp14 
Phage ID: Turbido-DRAFT 
Phage Name: Turbido-DRAFT 
Phage Accession: 
HostStrin: Not Found
********************************
Mycobacterium phage D29
********************************             
Gene ID: 28369_14 
Phage ID: 28369 
Phage Name: D29 
Phage Accession: AF022214.1
HostStrin: Mycobacterium phage D29
********************************
Mycobacterium phage Che12
********************************             
Gene ID: 373405_14 
Phage ID: 373405 
Phage Name: Che12 
Phage Accession: DQ398043.1
HostStrin: Myco

In [ ]:
phage = 'George-DRAFT'
print(phage.split('-')[0])